In [ ]:
!pip install pygame

In [ ]:
import pygame
import sys
import random
import time

In [ ]:
#defining basic game constants
WIDTH,HEIGHT=800,600
BALL_R=13
PFOM_WD,PFOM_HT=100,10
FPS=60
BLACK=(0,0,0)
WHITE=(255,255,255)
RED=(255,0,0)
YELLOW=(255,255,0)
GREEN=(150,255,120)
BLUE=(160,225,255)
GRAY=(80,80,80)
LG=(200,200,200)
GOLD=(220,220,55)

In [ ]:
#GAME INIT
pygame.init()

#INSTANCE WINDOW
window=pygame.display.set_mode([WIDTH,HEIGHT])
font=pygame.font.Font(None,30)

#SETTING CLOCK
clock=pygame.time.Clock()

#GAME CONSTANTS
b_pos=[ WIDTH // 2 , HEIGHT // 2 ]
b_speed=[random.uniform(2,4),random.uniform(2,4)]

pf_pos=[ (WIDTH-PFOM_WD) // 2 , HEIGHT-PFOM_HT-10 ]
pf_speed=10
score=0
level=1
pf_color=BLUE
hearts=3
b_color=WHITE

In [ ]:
#OPERATIONAL FUNCTIONALS  
def getch():
    ch_state=True
    while ch_state:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            else:
                ch_state=False

In [ ]:
def printscreen(text,size,pos,clr=WHITE):
    font = pygame.font.Font(None, size)
    text_render = font.render(text, True, clr)
    text_box = text_render.get_rect(center=(WIDTH // 2, HEIGHT-pos))
    window.blit(text_render, text_box)

In [ ]:
def startscreen():
    window.fill(BLACK)
    printscreen("BOUNCE",65,HEIGHT//2)
    printscreen("PRESS ANY KEY TO START...", 25, HEIGHT // 7)
    printscreen("USE ARROWS TO MOVE THE PLATFORM...", 25, HEIGHT // 5)
    pygame.display.flip()
    getch()

In [ ]:
def endscreen():
    window.fill(GRAY)
    printscreen("GAME OVER",65,HEIGHT//2,RED)
    printscreen("PRESS ANY KEY TO START...", 25, HEIGHT // 7)
    printscreen("USE ARROWS TO MOVE THE PLATFORM...", 25, HEIGHT // 5)
    pygame.display.flip()
    getch()

In [ ]:

def winscreen():
    window.fill(LG)
    printscreen("CONGRATULATIONS!!!",65,HEIGHT//2,GOLD)
    printscreen("YOU WIN!!! NOW GET A LIFE!!!", 25, HEIGHT // 5,GOLD)
    printscreen("PRESS ANY KEY TO END OR ANY OTHER KEY TO RESTART...", 25, HEIGHT // 5)
    pygame.display.flip()
    getch()

In [ ]:
def color():
    return [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]

In [ ]:
def levelup():
        global b_speed,b_color,b_pos,pf_color,pf_pos,pf_speed,level,hearts
        hearts+=1
        level += 1
        b_pos = [WIDTH // 2, HEIGHT // 2]
        b_speed = [random.uniform(2, 4), random.uniform(2, 4)]
        pf_speed=random.uniform(5,20)
        pf_color,b_color=color(),color()
        return 

In [ ]:
def move_paddle(keys):
    global b_speed,b_color,b_pos,pf_color,pf_pos,pf_speed

    # MOVE THE PADDLE
    pf_pos[0] += (keys[pygame.K_RIGHT] - keys[pygame.K_LEFT]) * pf_speed
    pf_pos[1] += (keys[pygame.K_DOWN] - keys[pygame.K_UP]) * pf_speed

    # KEEPING THE PADDLE WITHIN SCREEN BOUNDS
    pf_pos[0] = max(0, min(pf_pos[0], WIDTH - PFOM_WD))
    pf_pos[1] = max(0, min(pf_pos[1], HEIGHT - PFOM_HT))
    return 

In [ ]:
def move_ball():
    
    global b_speed,b_color,b_pos,pf_color,pf_pos,pf_speed

    #UPDATE BALL POSITION
    b_pos[0] += b_speed[0]
    b_pos[1] += b_speed[1]

    # BALL BOUNCEBACK
    if b_pos[0] <= 0 or b_pos[0] >= WIDTH:
        b_speed[0] = -b_speed[0]

    if b_pos[1] <= 0:
        b_speed[1] = -b_speed[1]
    
    return b_pos,b_speed

In [ ]:
def keep_score():

    global b_speed,b_color,b_pos,pf_color,pf_pos,pf_speed,level,hearts,score

    # BALL PADDLE CONTACT TEST
    if (pf_pos[0] <= b_pos[0] <= pf_pos[0] + PFOM_WD and pf_pos[1] <= b_pos[1] <= pf_pos[1] + PFOM_HT):
        b_speed[1] = - b_speed[1]
        score += 1
    
    if score >= level * 10:
        levelup()
    
    if b_pos[1] >= HEIGHT:
        # DEAD
        hearts -= 1
        if hearts <= 0:
            endscreen()
            score = 0
            heart = 3
            level = 1
        else:
            b_pos = [WIDTH // 2, HEIGHT // 2]
            b_speed = [random.uniform(random.uniform(1,4), random.uniform(4,8)), random.uniform(random.uniform(1,4), random.uniform(4,8))]*2

In [ ]:
#DRAW GAME ELEMENTS
def draw():
    
    global b_speed,b_color,b_pos,pf_color,pf_pos,pf_speed,font

    pygame.draw.circle(window, b_color, (int(b_pos[0]), int(b_pos[1])), BALL_R)
    pygame.draw.rect(window, pf_color, (int(pf_pos[0]), int(pf_pos[1]), PFOM_WD, PFOM_HT))
    
    info_pos = 10  
    spacing = 75  
    # DISPLAY SCORE
    score_txt = font.render(f"Score: {score}", True, WHITE)
    score_box = score_txt.get_rect(topleft=(10, info_pos))
    pygame.draw.rect(window, GOLD, score_box.inflate(10, 5))
    window.blit(score_txt, score_box)

    # LEVEL INDICATOR
    lvl_txt = font.render(f"Level: {level}", True, WHITE)
    lvl_box = lvl_txt.get_rect(topleft=(score_box.topright[0] + spacing, info_pos))
    pygame.draw.rect(window, BLUE, lvl_box.inflate(10, 5))
    window.blit(lvl_txt, lvl_box)

    # Draw the lives in a red rectangle at the top left (next to the level)
    life_txt = font.render(f"Lives: {hearts}", True, WHITE)
    life_box = life_txt.get_rect(topleft=(lvl_box.topright[0] + spacing, info_pos))
    pygame.draw.rect(window, RED, life_box.inflate(10, 5))
    window.blit(life_txt, life_box)


In [ ]:
#GAME LOOP
startscreen()
run_state=True
while run_state:
    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            run_state=False
    
    keys = pygame.key.get_pressed()
    
    move_paddle(keys)
    move_ball()
    keep_score()
    #CLEARSCREEN
    window.fill(BLACK)
    draw()
    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
    